In [4]:
import os
os.chdir('..')

In [2]:
import orjson
from pathlib import Path

In [6]:
original_path = Path('data/predict/example03.jsonl/few_shot_with_reason_3_ft:gpt-3.5-turbo-0613:personal:100-withr-3:85EQuAFS.jsonl')
original = [orjson.loads(i) for i in original_path.read_text().split('\n')[:-1]]

print(len(original))
original[-1]

5155


{'pred': 1,
 'reason': 'The news is about a cartoon dog and a political party in Hungary, which is not related to Taiwan and does not provide relevant economic information for constructing EPU index.'}

In [9]:
raw_path = Path('data/raw/example03.jsonl')
raw = [orjson.loads(i) for i in raw_path.read_text().split('\n')[:-1]]
target = raw[len(original):]

print(len(target))
raw[0]

1920


{'label': 1,
 'news': '美國本屆總統大選到選前最後一刻仍呈拉鋸緊繃狀態，專家又普遍預期結果將取決於幾個關鍵的搖擺州，此一情況不免讓各界憂心2000年美國總統大選爆發驗票爭議選舉結果一延再延的噩夢可能重演。尋求連任的歐巴馬民調支持度始終無法甩開對手羅姆尼的緊追，前不久甚至還一度出現遭超前的情況，最近雖告反彈，但羅姆尼依然咬住，選前最後民調指向歐巴馬微幅領先，羅姆尼不無可能最後衝出逆轉勝。由於選情高度緊繃，兩大陣營都對外宣稱可贏得大選，但同時也預見可能出現選舉爭議，因此也都為可能的訴訟預作準備。歐巴馬與羅姆尼的競選團隊均已組成法律顧問小組，因應選後可能因投票爭議與重新驗票的選舉官司。回顧2000年，小布希與民主黨候選人高爾對決，2人民調支持度也是呈現拉鋸，最後開票高爾普選票贏小布希50多萬票，但選舉人票則不相上下，在搖擺州佛羅里達2人普選票得票率更是旗鼓相當，因而爆發計票問題與重新驗票爭議。由於佛羅里的州選舉結果攸關最後勝負誰屬，雙方你來我往進行選舉訴訟，官司打了一個多月，最後才由最高法院裁奪小布希在佛州得票數小贏高爾537票，拿下該州25張選舉人票，最後以271張選舉人票驚險入主白宮。選舉結果未定久拖是對經濟與金融市場最不利的情況，2000年美國總統大選驗票爭議耗時逾一個月，不確定性不但造成社會動盪不安，還拖累美股全面重挫5％左右。專家指出，今年美國總統大選確實有可能出現無法在當天決定結果的情況，以搖擺州俄亥俄州為例，該州不在籍投票超過140萬張，但這些選民也可變更為到投票所投下「臨時票」，為避免重複計票，該州規定臨時票16日後才計票，意謂著若2黨得票數相當接近，且該州又是決定勝負結果的關鍵，則總統大選結果就會出現難產。'}

In [11]:
target_save_path = Path('data/raw/example03_target.jsonl')
with target_save_path.open('wb') as f:
    for i in target:
        f.write(orjson.dumps(i, option = orjson.OPT_APPEND_NEWLINE))

In [11]:
target_pred_path = Path('data/predict/example03_target.jsonl/few_shot_with_reason_3_ft:gpt-3.5-turbo-0613:personal:100-withr-3:85EQuAFS.jsonl')
target_pred = [orjson.loads(i) for i in target_pred_path.read_text().split('\n')[:-1]]

merge = original + target_pred

10007

In [ ]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
import mlflow
import matplotlib.pyplot as plt
from langchain.callbacks.manager import get_openai_callback


def log_predict(cb, news, labels, pred, reason):
    metric_dict = classification_report(labels, pred, output_dict = True)
    table_dict = (
        {
            "news": news,
            "reason": reason,
            "pred": pred,
            "label": labels,
        }
        if self.strategy in ["zero_shot_with_reason", "few_shot_with_reason"]
        else
        {
            "news": news,
            "pred": pred,
            "label": labels,
        }
    )

    exper = mlflow.set_experiment(f"EPU denoise_{self.input_name}")
    with mlflow.start_run(
        experiment_id = exper.experiment_id,
        run_name = "_".join((self.strategy, str(self.n_example), self.model_name)),
    ):
        mlflow.log_param("model", self.model_name)
        mlflow.log_param("few_shot_n_example", self.n_example)
        mlflow.log_param("used_tokens", cb.total_tokens)
        mlflow.log_param("fine_tune_strategy", self.strategy)

        if self.model_name[:2] == 'ft':
            mlflow.log_param("epochs", self.model_name.split(':')[-2].split('-')[2])
            mlflow.log_param("fine_tune_n_example", self.model_name.split(':')[-2].split('-')[0])
            cost = 0.012*(cb.prompt_tokens/1000) + 0.016*(cb.completion_tokens/1000)
            mlflow.log_param("cost", cost)
        else:
            mlflow.log_param("cost", cb.total_cost)

        mlflow.log_metric("precision_0", metric_dict["0"]["precision"])
        mlflow.log_metric("micro_f1", metric_dict["accuracy"])
        mlflow.log_metric("macro_f1", metric_dict["macro avg"]["f1-score"])
        mlflow.log_metric("weighted_f1", metric_dict["weighted avg"]["f1-score"])

        cm = confusion_matrix(labels, pred, normalize="pred")
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.savefig("confustion_matrix.png")

        mlflow.log_artifact("confustion_matrix.png")
        mlflow.log_table(table_dict, "news_pred_reason.json")

In [12]:
labels = [i.get('label') for i in raw]
news = [i.get('news') for i in raw]
pred = [i.get('pred') for i in merge]
reason = [i.get('reason') for i in merge]

with get_openai_callback() as cb:
    log_predict(cb, news, labels, pred, reason)

[{'label': 1,
  'news': '美國本屆總統大選到選前最後一刻仍呈拉鋸緊繃狀態，專家又普遍預期結果將取決於幾個關鍵的搖擺州，此一情況不免讓各界憂心2000年美國總統大選爆發驗票爭議選舉結果一延再延的噩夢可能重演。尋求連任的歐巴馬民調支持度始終無法甩開對手羅姆尼的緊追，前不久甚至還一度出現遭超前的情況，最近雖告反彈，但羅姆尼依然咬住，選前最後民調指向歐巴馬微幅領先，羅姆尼不無可能最後衝出逆轉勝。由於選情高度緊繃，兩大陣營都對外宣稱可贏得大選，但同時也預見可能出現選舉爭議，因此也都為可能的訴訟預作準備。歐巴馬與羅姆尼的競選團隊均已組成法律顧問小組，因應選後可能因投票爭議與重新驗票的選舉官司。回顧2000年，小布希與民主黨候選人高爾對決，2人民調支持度也是呈現拉鋸，最後開票高爾普選票贏小布希50多萬票，但選舉人票則不相上下，在搖擺州佛羅里達2人普選票得票率更是旗鼓相當，因而爆發計票問題與重新驗票爭議。由於佛羅里的州選舉結果攸關最後勝負誰屬，雙方你來我往進行選舉訴訟，官司打了一個多月，最後才由最高法院裁奪小布希在佛州得票數小贏高爾537票，拿下該州25張選舉人票，最後以271張選舉人票驚險入主白宮。選舉結果未定久拖是對經濟與金融市場最不利的情況，2000年美國總統大選驗票爭議耗時逾一個月，不確定性不但造成社會動盪不安，還拖累美股全面重挫5％左右。專家指出，今年美國總統大選確實有可能出現無法在當天決定結果的情況，以搖擺州俄亥俄州為例，該州不在籍投票超過140萬張，但這些選民也可變更為到投票所投下「臨時票」，為避免重複計票，該州規定臨時票16日後才計票，意謂著若2黨得票數相當接近，且該州又是決定勝負結果的關鍵，則總統大選結果就會出現難產。'},
 {'label': 0,
  'news': '記者王憶紅／專題報導貨櫃航運雖然進入年底淡季，但在全球最大貨櫃航運業者麥司克航運（Maersk）與地中海全球前2大貨櫃船公司宣布將自12月15日開徵旺季附加費，每FEU（40呎）收400美元，加上長榮集團力挺旗下長榮海（2603），貨櫃航運上週後半週再展開一波攻勢。法人預期，貨櫃航運雖有利多題材加持，但因基本面獲利狀況仍處淡季，個股走勢仍以震盪居多。全球2大貨櫃船公司宣布開徵旺季附加費，無異是為貨櫃航運市場打了一劑強心針，後續不排除多家船公司也將跟進。不過，法人指出，最